In [ ]:
# 1. INSTALL THE LIBRARIES (Needed if you haven't done it yet!)
!pip install openai gradio

# Imports
from openai import OpenAI
import os
import gradio as gr # Import the Gradio library

# 2. API KEY SETUP
# Load API key from environment variable for security
# Set your API key as an environment variable: export OPENAI_API_KEY="your-key-here"
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY environment variable is not set. Please set it before running this notebook.")

# 3. INITIALIZE THE CLIENT
try:
    client = OpenAI(api_key=api_key)
    print("✔️ OpenAI Client Initialized.")
except Exception as e:
    print(f"ERROR: Could not initialize OpenAI client. Please check your API key. Details: {e}")
    # In a notebook, this is a soft exit.
    exit()

# 4. DEFINE THE DATABASE CONTEXT (CRUCIAL for SQL generation)
# This is the schema information the AI will use to write the query.
database_schema = """
-- Table: Products
-- Columns: ProductID (INT, PK), Name (TEXT), Price (REAL), Category (TEXT)

-- Table: Orders
-- Columns: OrderID (INT, PK), CustomerID (INT, FK), OrderDate (TEXT), TotalAmount (REAL)

-- Table: Customers
-- Columns: CustomerID (INT, PK), FirstName (TEXT), LastName (TEXT), Email (TEXT), State (TEXT)
"""

# 6. DEFINE THE SYSTEM PROMPT (The AI's Rules)
# This rarely changes, so it belongs in the setup cell.
system_prompt = f"""
You are an expert SQL translator. Your sole purpose is to convert natural language questions
into a single, correct SQL query for a SQLite database.

DO NOT output any explanation, surrounding text, or markdown formatting (like ```sql).
Only output the raw SQL query.

Here is the database schema you MUST use:
{database_schema}
"""

✔️ OpenAI Client Initialized.


In [2]:
# 2: THE CORE GENERATION FUNCTION
def generate_sql_query(question):
    """
    Sends the user's question and the system prompt to the OpenAI API
    and returns the generated SQL query string.
    """
    
    # Construct the full list of messages for the API call
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
    ]

    try:
        print(f"-> Sending query to GPT-3.5-turbo...")
        # 7. MAKE THE API CALL
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0.0, # Low temp is best for coding/factual tasks
            max_tokens=150   # Limit the length
        )

        # 8. EXTRACT AND RETURN THE GENERATED SQL
        generated_sql = response.choices[0].message.content.strip()
        
        # Gradio can display the output cleanly
        return generated_sql

    except Exception as e:
        error_message = f"API CALL FAILED: {e}. Check key and network connection."
        print(error_message)
        # Return the error message to the Gradio output box
        return error_message

In [3]:
# 3: START THE GRADIO UI (Run this cell to launch the app) ---

# Define the Gradio interface
iface = gr.Interface(
    # The function to call when the user submits a question
    fn=generate_sql_query, 
    
    # The input component: a text box for the user's natural language question
    inputs=gr.Textbox(label="Ask a Natural Language Question:", 
                      placeholder="e.g., Show me all customer names from California.", 
                      lines=3),
    
    # The output component: a text box to display the generated SQL query
    outputs=gr.Textbox(label="Generated SQL Query (Copyable)", 
                       lines=5),
    
    # Title and description for the web app
    title="💣 SQL Query Buddy (OpenAI + Gradio)",
    description="Type a question based on the provided schema, and the AI will generate the corresponding SQL query.",
    
    # Additional components for context (optional but helpful)
    allow_flagging="never", # Recommended for simple demos
)

# Launch the Gradio application
# This will display a local link (and a Public URL if run on a cloud environment)
iface.launch(inbrowser=True)

/home/unit-s/my_env/lib/python3.12/site-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


-> Sending query to GPT-3.5-turbo...
